# Inside AirBnB analysis - Berlin, Germany

Data source: http://insideairbnb.com/get-the-data.html

Aim: pratice use of sqlite while exploring AirBnB data for Berlin, Germany.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
import folium
from folium.features import DivIcon

In [2]:
sqlite_file = 'AirBnB_Berlin.db'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

### 1. What's the most expensive listing? What else can you tell me about the listing?
<br>
Answer: 9000 is the most expensive listing. The listing is an entire home in Charlottenburg-Wilm. located in the Duesseldorfer Str. The owners name is David. According to the data the place was always booked. It received the last review in January 2016.

In [3]:
for row in c.execute('SELECT *,max(price) FROM listings'):
    print(row)

(9651871, 'Über den Dächern Berlins Zentrum', 49896804, 'David', 'Charlottenburg-Wilm.', 'Düsseldorfer Straße', 52.49797795015381, 13.334436985012973, 'Entire home/apt', 9000, 100, 1, '2016-01-02', '0.06', 1, '0', 9000)


### 2. What neighbourhoods seem to be the most popular?
<br>
Answer: Most popular neighbourhoods are Neukoelln and Friedrichshain-Kreuzberg.

In [4]:
c.execute('SELECT neighbourhood_group, AVG(availability_365) avg_availability, AVG(lat) lat, AVG(long)  FROM listings GROUP BY 1 order by avg_availability')
result=c.fetchall()
Q2df = pd.DataFrame(result)
#df.columns="0": "Neighbourhood", "1": "Availability", "2": "lat", "3":"long"})
Q2df.columns = ['Neighbourhood', 'Availability', 'Lat', 'Long']
print(Q2df)


               Neighbourhood  Availability        Lat       Long
0                   Neukölln     79.778966  52.478091  13.435388
1   Friedrichshain-Kreuzberg     91.637348  52.503904  13.434093
2                      Mitte    108.459328  52.532835  13.377992
3                     Pankow    109.187333  52.545055  13.421486
4                Lichtenberg    110.887324  52.512116  13.493122
5     Tempelhof - Schöneberg    116.618764  52.481452  13.358401
6         Treptow - Köpenick    118.093333  52.462406  13.514383
7       Charlottenburg-Wilm.    140.800892  52.503356  13.306923
8      Steglitz - Zehlendorf    152.781333  52.444193  13.293232
9              Reinickendorf    160.304933  52.582231  13.328510
10                   Spandau    174.490196  52.529326  13.194536
11     Marzahn - Hellersdorf    221.365854  52.517985  13.578544


In [5]:
#Make a map of the neighbourhoods
folium_map = folium.Map(location=[52.52, 13.405], zoom_start=11)

#NK = folium.CircleMarker(location=[Q2df.iloc[0][2], Q2df.iloc[0][3]])
#NK.add_to(folium_map)
FH_K = folium.CircleMarker(location=[Q2df.iloc[1][2], Q2df.iloc[1][3]],icon=DivIcon(icon_size=(150,36),icon_anchor=(0,0),html='<div style="font-size: 24pt">Test</div>',))
#Label does not work 
FH_K.add_to(folium_map)

In [12]:
for index, row in Q2df.iterrows():
    Availability = row["Availability"]
    radius =  1/Availability * 3000
    #if net_departures>0:
    color="#E37222" # tangerine
    #tooltip = row["Neighbourhood"]
    popup=row["Neighbourhood"]
    folium.CircleMarker(location=(row["Lat"],
                                  row["Long"]),
                        radius=radius,
                        color=color,
                        popup=popup,
                        #tooltip = tooltip,
                        fill=True).add_to(folium_map)

In [13]:
folium_map

### 3. What time of the year is the cheapest time to go to your city? 
<br>
Answer: 
<br>
Assuming that the cheapest time of the year is when the cheapest places are available and further assuming that availability is defined as the time when the cheapest places are receiving the lowest amount of reviews, the cheapest time to visit Berlin is January.


In [ ]:
# What is the price range
for row in c.execute('SELECT min(price) as min_price, max(price) as max_price, avg(price) FROM listings'):
    print(row)
# Definition everything below mean is cheap

In [ ]:
c.execute('with MonthlyReviews As(SELECT listing_id,count(listing_id) as reviews_total, strftime("%m", Date) as ''Month'' FROM reviews group by 1,3), CheapPlaces AS (SELECT listings.id, listings.price FROM listings WHERE listings.price < 58 ) SELECT MonthlyReviews.Month, AVG(MonthlyReviews.reviews_total) as reviews_total FROM CheapPlaces JOIN MonthlyReviews ON CheapPlaces.id = MonthlyReviews.listing_id GROUP BY 1')
totalreviewsofcheapplacespermonth= c.fetchall()
#print(totalreviewsofcheapplacespermonth)

df = pd.DataFrame(totalreviewsofcheapplacespermonth)
df.columns = ['Month','reviews_total']
df[['Month','reviews_total']].set_index('Month').plot()

#print(df)
plt.legend(loc='lower right') 
plt.xlabel('Month')
plt.ylabel('Number of reviews total')
plt.title('Average Number of reviews of places that cost less than 58\n')

### 4. What about the busiest?
<br>
Answer: Calculating from the average over data from year 2009 till 2017, the busiest month is April, closely followed by September.
<br>
Plot number of reviews over time.

In [ ]:
c.execute('SELECT Count(listing_id) as reviews_total, strftime("%m", Date) as ''Month'' FROM reviews group by strftime("%m", Date)')
result = c.fetchall()

df = pd.DataFrame(result)
df.columns = ['reviews_total', 'Month']
df[['Month','reviews_total']].set_index('Month').plot()

plt.legend(loc='upper right') 
plt.xlabel('Month')
plt.ylabel('Number of reviews total')
plt.title('What is the busiest time of year?\n')
